In [1]:
import pandas as pd
import os

In [2]:
RAW_DATA_PATH = '../raw/'
def read_all_csv_files():
    """
    Lee todos los archivos CSV del directorio raw
    """
    
    dataframes = {}

    for file in os.listdir(RAW_DATA_PATH):
        if file.endswith(".csv"):
            table_name = file.replace(".csv", "")
            file_path = os.path.join(RAW_DATA_PATH, file)

            try:
                df = pd.read_csv(file_path)
                dataframes[table_name] = df
                print(f"{file} was successfully read. Rows: {len(df)}")
            except Exception as e:
                print(f"Error: {file} | {str(e)}")
                raise
        
    return dataframes
raw_data = read_all_csv_files()

campaigns.csv was successfully read. Rows: 12
categories.csv was successfully read. Rows: 12
channels.csv was successfully read. Rows: 4
customers.csv was successfully read. Rows: 250
customer_addresses.csv was successfully read. Rows: 394
inventory.csv was successfully read. Rows: 300
orders.csv was successfully read. Rows: 400
order_items.csv was successfully read. Rows: 964
order_marketing.csv was successfully read. Rows: 400
payments.csv was successfully read. Rows: 399
products.csv was successfully read. Rows: 150
shipments.csv was successfully read. Rows: 209
warehouses.csv was successfully read. Rows: 2


In [3]:
campaigns = raw_data["campaigns"]
channels = raw_data["channels"]
orders = raw_data["orders"]
order_item = raw_data["order_items"]
products = raw_data["products"]
categories = raw_data["categories"]
order_marketing = raw_data["order_marketing"]

In [4]:
categories_with_parents = pd.merge(
    categories,
    categories.add_prefix("parent_"),
    on = "parent_category_id",
    how = "left",
)[["category_id","category_name","parent_category_name"]]
categories_with_parents

,category_id,category_name,parent_category_name
0,1,Ropa,NaN
1,2,Calzado,NaN
2,3,Accesorios,NaN
3,4,Electrónica,NaN
4,5,Camperas,Ropa
5,6,Remeras,Ropa
6,7,Pantalones,Ropa
7,8,Zapatillas,Calzado
8,9,Botas,Calzado
9,10,Mochilas,Accesorios


In [5]:
dim_product = pd.merge(
    products,
    categories_with_parents,
    on = "category_id",
    how = "left"
).drop(columns = ['category_id'])
dim_product

,product_id,sku,product_name,brand,unit_price,unit_cost,active_from,active_to,created_at,category_name,parent_category_name
0,1,SKU-00001,Andina Remeras 001,Andina,112524.82,68777.76,2025-02-05,NaN,2025-02-05 03:43:54,Remeras,Ropa
1,2,SKU-00002,Lago Remeras 002,Lago,9608.49,5509.74,2025-03-01,NaN,2025-03-01 18:23:57,Remeras,Ropa
2,3,SKU-00003,Lago Zapatillas 003,Lago,36963.89,25775.53,2025-07-27,NaN,2025-07-27 00:14:11,Zapatillas,Calzado
3,4,SKU-00004,Lago Pantalones 004,Lago,54336.32,31997.02,2025-07-15,NaN,2025-07-15 12:15:18,Pantalones,Ropa
4,5,SKU-00005,Pampa Gorras 005,Pampa,57052.01,36284.45,2025-03-09,NaN,2025-03-09 01:34:55,Gorras,Accesorios
...,...,...,...,...,...,...,...,...,...,...,...
145,146,SKU-00146,Lago Pantalones 146,Lago,70778.34,50792.07,2025-07-10,NaN,2025-07-10 19:40:04,Pantalones,Ropa
146,147,SKU-00147,Lago Pantalones 147,Lago,99963.68,75814.78,2025-06-07,2025-07-27,2025-06-07 14:51:32,Pantalones,Ropa
147,148,SKU-00148,Norte Mochilas 148,Norte,69290.55,54575.92,2025-04-03,2025-07-21,2025-04-03 03:37:02,Mochilas,Accesorios
148,149,SKU-00149,Andina Mochilas 149,Andina,62730.25,37478.32,2025-02-01,NaN,2025-02-01 16:33:20,Mochilas,Accesorios


In [6]:
order_item.isnull().sum()

order_id           0
line_number        0
product_id         0
quantity           0
unit_price         0
discount_amount    0
tax_amount         0
dtype: int64

In [7]:
order_item_prod = pd.merge(
    order_item,
    dim_product,
    on = 'product_id',
    how = 'left',
    suffixes = ['','_product']
)
order_item_prod.head()

,order_id,line_number,product_id,quantity,unit_price,discount_amount,tax_amount,sku,product_name,brand,unit_price_product,unit_cost,active_from,active_to,created_at,category_name,parent_category_name
0,100001,1,137,2,97309.59,13623.34,38009.13,SKU-00137,Norte Mochilas 137,Norte,97309.59,68624.94,2025-06-23,NaN,2025-06-23 04:58:47,Mochilas,Accesorios
1,100001,2,148,2,69290.55,2771.62,28519.99,SKU-00148,Norte Mochilas 148,Norte,69290.55,54575.92,2025-04-03,2025-07-21,2025-04-03 03:37:02,Mochilas,Accesorios
2,100001,3,53,3,55916.19,8387.43,33465.84,SKU-00053,Estilo Remeras 053,Estilo,55916.19,37874.91,2025-06-23,NaN,2025-06-23 19:31:22,Remeras,Ropa
3,100001,4,77,3,35690.74,12848.67,19786.95,SKU-00077,Río Gorras 077,Río,35690.74,22546.25,2025-07-14,NaN,2025-07-14 13:48:12,Gorras,Accesorios
4,100002,1,140,1,42260.66,5071.28,7809.77,SKU-00140,Sur Botas 140,Sur,42260.66,31956.35,2025-05-04,NaN,2025-05-04 10:44:44,Botas,Calzado


In [8]:
# (order_item_prod['unit_price'] == order_item_prod['unit_price_product']).value_counts()
# unit_price columns are the same, drop
order_item_prod = order_item_prod.drop(columns = ['product_id', 'unit_price_product'])
order_item_prod.head()

,order_id,line_number,quantity,unit_price,discount_amount,tax_amount,sku,product_name,brand,unit_cost,active_from,active_to,created_at,category_name,parent_category_name
0,100001,1,2,97309.59,13623.34,38009.13,SKU-00137,Norte Mochilas 137,Norte,68624.94,2025-06-23,NaN,2025-06-23 04:58:47,Mochilas,Accesorios
1,100001,2,2,69290.55,2771.62,28519.99,SKU-00148,Norte Mochilas 148,Norte,54575.92,2025-04-03,2025-07-21,2025-04-03 03:37:02,Mochilas,Accesorios
2,100001,3,3,55916.19,8387.43,33465.84,SKU-00053,Estilo Remeras 053,Estilo,37874.91,2025-06-23,NaN,2025-06-23 19:31:22,Remeras,Ropa
3,100001,4,3,35690.74,12848.67,19786.95,SKU-00077,Río Gorras 077,Río,22546.25,2025-07-14,NaN,2025-07-14 13:48:12,Gorras,Accesorios
4,100002,1,1,42260.66,5071.28,7809.77,SKU-00140,Sur Botas 140,Sur,31956.35,2025-05-04,NaN,2025-05-04 10:44:44,Botas,Calzado


In [9]:
campaigns_2 = campaigns.drop(columns = ['channel_id', 'utm_medium', 'utm_source', 'utm_campaign'])
order_marketing_campaign = pd.merge(
    order_marketing,
    campaigns_2,
    on = 'campaign_id',
    how = 'left',
).drop(columns = ['campaign_id'])
order_marketing_campaign.head()

,order_id,channel_id,utm_source,utm_medium,utm_campaign,utm_content,utm_term,campaign_name,start_date,end_date,budget_ars
0,100001,1,meta,social,retargeting_jul_2025,NaN,mochilas,Retargeting Julio 2025 - Web,2025-08-08,2025-08-27,3157411.81
1,100002,3,meta,social,retargeting_jul_2025,remarketing,remeras,Retargeting Julio 2025 - Marketplace,2025-08-16,2025-09-02,2997894.66
2,100003,3,meta,social,nuevo_drop_ago_2025,remarketing,camperas,Nuevo Drop Agosto 2025 - Marketplace,2025-07-27,2025-08-12,4801475.51
3,100004,1,email,email,newsletter,NaN,camperas,NaN,NaN,NaN,NaN
4,100005,3,meta,social,nuevo_drop_ago_2025,promo,mochilas,Nuevo Drop Agosto 2025 - Marketplace,2025-07-27,2025-08-12,4801475.51


In [10]:
order_marketing_campaign_channel = pd.merge(
    order_marketing_campaign,
    channels,
    on = 'channel_id',
    how = 'left'
).drop(columns = ['channel_id'])
order_marketing_campaign_channel.head()

,order_id,utm_source,utm_medium,utm_campaign,utm_content,utm_term,campaign_name,start_date,end_date,budget_ars,channel_name,description
0,100001,meta,social,retargeting_jul_2025,NaN,mochilas,Retargeting Julio 2025 - Web,2025-08-08,2025-08-27,3157411.81,Web,Tienda online desktop/mobile
1,100002,meta,social,retargeting_jul_2025,remarketing,remeras,Retargeting Julio 2025 - Marketplace,2025-08-16,2025-09-02,2997894.66,Marketplace,Marketplace externo
2,100003,meta,social,nuevo_drop_ago_2025,remarketing,camperas,Nuevo Drop Agosto 2025 - Marketplace,2025-07-27,2025-08-12,4801475.51,Marketplace,Marketplace externo
3,100004,email,email,newsletter,NaN,camperas,NaN,NaN,NaN,NaN,Web,Tienda online desktop/mobile
4,100005,meta,social,nuevo_drop_ago_2025,promo,mochilas,Nuevo Drop Agosto 2025 - Marketplace,2025-07-27,2025-08-12,4801475.51,Marketplace,Marketplace externo


In [11]:
orders.shape

(400, 14)

In [12]:
import datetime as dt
start_date = pd.to_datetime(orders["order_date"]).min()
end_date = pd.to_datetime(orders["order_date"]).max()
print(start_date, end_date)

2025-06-01 06:07:52 2025-08-31 21:46:17


In [13]:
date_range = pd.date_range(start=start_date, end=end_date, freq="D").normalize()

# Step 2: Build dataframe
dim_date = pd.DataFrame({
    "date_key": date_range.strftime("%Y%m%d").astype(int),   # surrogate key in YYYYMMDD
    "full_date": date_range,
    "year": date_range.year,
    "quarter": date_range.quarter,
    "month": date_range.month,
    "month_name": date_range.month_name(),
    "week_of_year": date_range.isocalendar().week,
    "day_of_month": date_range.day,
    "day_of_week": date_range.weekday + 1,   # Monday=1 ... Sunday=7
    "day_name": date_range.day_name(),
    "is_weekend": date_range.weekday >= 5    # Saturday=5, Sunday=6
})
dim_date

,date_key,full_date,year,quarter,month,month_name,week_of_year,day_of_month,day_of_week,day_name,is_weekend
2025-06-01,20250601,2025-06-01,2025,2,6,June,22,1,7,Sunday,True
2025-06-02,20250602,2025-06-02,2025,2,6,June,23,2,1,Monday,False
2025-06-03,20250603,2025-06-03,2025,2,6,June,23,3,2,Tuesday,False
2025-06-04,20250604,2025-06-04,2025,2,6,June,23,4,3,Wednesday,False
2025-06-05,20250605,2025-06-05,2025,2,6,June,23,5,4,Thursday,False
...,...,...,...,...,...,...,...,...,...,...,...
2025-08-27,20250827,2025-08-27,2025,3,8,August,35,27,3,Wednesday,False
2025-08-28,20250828,2025-08-28,2025,3,8,August,35,28,4,Thursday,False
2025-08-29,20250829,2025-08-29,2025,3,8,August,35,29,5,Friday,False
2025-08-30,20250830,2025-08-30,2025,3,8,August,35,30,6,Saturday,True


In [14]:
time_range = pd.date_range("00:00:00", "23:59:59", freq="1min").time
dim_time = pd.DataFrame({
    "time_key": range(1, len(time_range)+1),  # surrogate key starting from 1
    "full_time": time_range,
    "hour": [t.hour for t in time_range],
    "minute": [t.minute for t in time_range],
    "second": [t.second for t in time_range],
    "am_pm": ["AM" if t.hour < 12 else "PM" for t in time_range],
})
dim_time

,time_key,full_time,hour,minute,second,am_pm
0,1,00:00:00,0,0,0,AM
1,2,00:01:00,0,1,0,AM
2,3,00:02:00,0,2,0,AM
3,4,00:03:00,0,3,0,AM
4,5,00:04:00,0,4,0,AM
...,...,...,...,...,...,...
1435,1436,23:55:00,23,55,0,PM
1436,1437,23:56:00,23,56,0,PM
1437,1438,23:57:00,23,57,0,PM
1438,1439,23:58:00,23,58,0,PM


In [15]:
orders['date'] = pd.to_datetime(orders['order_date']).dt.date.astype(str)
orders['time'] = pd.to_datetime(orders['order_date']).dt.floor('min').dt.time.astype(str)

In [16]:
dim_date['full_date'] = dim_date['full_date'].astype(str)
dim_time['full_time'] = dim_time['full_time'].astype(str)

In [17]:
fact_table = pd.merge(
    orders,
    dim_date[['date_key','full_date']],
    left_on = 'date',
    right_on = 'full_date'
).drop(columns = ['full_date','date'])

fact_table_2 = pd.merge(
    fact_table,
    dim_time[['time_key','full_time']],
    left_on = 'time',
    right_on = 'full_time',
).drop(columns = ['full_time','time', 'order_date'])

fact_table_2 = fact_table_2.rename(columns={
    'date_key': 'order_date_id',
    'time_key': 'order_time_id',
})

fact_table_3 = fact_table_2.copy()
fact_table_3['created_at_date'] = pd.to_datetime(fact_table_3['created_at']).dt.date.astype(str)
fact_table_3['created_at_time'] = pd.to_datetime(fact_table_3['created_at']).dt.floor('min').dt.time.astype(str)

In [18]:
fact_table_4 = pd.merge(
    fact_table_3,
    dim_date[['date_key','full_date']],
    left_on = 'created_at_date',
    right_on = 'full_date'
).drop(columns = ['full_date','created_at_date'])

fact_table_5 = pd.merge(
    fact_table_4,
    dim_time[['time_key','full_time']],
    left_on = 'created_at_time',
    right_on = 'full_time',
).drop(columns = ['full_time','created_at_time', 'created_at'])

fact_table_5 = fact_table_5.rename(columns={
    'date_key': 'created_at_date_id',
    'time_key': 'created_at_time_id',
})
fact_table_5

,order_id,customer_id,order_status,channel_id,billing_address_id,shipping_address_id,currency,shipping_amount,order_subtotal,order_discount,order_tax,order_total,order_date_id,order_time_id,created_at_date_id,created_at_time_id
0,100001,228,delivered,1,359.0,358,ARS,0.0,608021.07,37631.06,119781.90,690171.91,20250824,326,20250824,326
1,100002,101,delivered,3,157.0,156,ARS,0.0,791961.63,42047.78,157481.91,907395.76,20250722,834,20250722,834
2,100003,99,paid,3,154.0,153,ARS,0.0,138581.10,16629.73,25609.79,147561.16,20250725,1321,20250725,1321
3,100004,9,shipped,1,NaN,14,ARS,0.0,125460.50,15055.26,23185.10,133590.34,20250830,806,20250830,806
4,100005,108,canceled,3,NaN,168,ARS,0.0,823628.91,82362.89,155665.86,896931.88,20250708,632,20250708,632
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,100396,159,delivered,3,242.0,241,ARS,0.0,255388.10,0.00,53631.50,309019.60,20250603,1365,20250603,1365
396,100397,17,delivered,2,25.0,24,ARS,0.0,412265.35,35573.92,79105.20,455796.63,20250819,645,20250819,645
397,100398,95,shipped,3,147.0,146,ARS,0.0,310203.67,13046.28,62403.05,359560.44,20250624,642,20250624,642
398,100399,53,paid,2,81.0,80,ARS,0.0,739301.85,8544.03,153459.14,884216.96,20250716,425,20250716,425


In [19]:
(fact_table_5.order_date_id == fact_table_5.created_at_date_id).value_counts()
# they are the same, lets drop one

True    400
Name: count, dtype: int64

In [20]:
fact_table_6 = fact_table_5.drop(columns = ['created_at_date_id','created_at_time_id'])
fact_table_6 = fact_table_6.rename(columns={'order_date_id':'date_id','order_time_id':'time_id'})
fact_table_6 = fact_table_6[[
    'order_id',
    'customer_id',
    'date_id',
    'time_id',
    'billing_address_id',
    'channel_id',
    'order_status',
    'currency',
    'shipping_amount',
    'order_subtotal',
    'order_discount',
    'order_tax',
    'order_total',
]]

In [21]:
fact_table_7 = pd.merge(
    fact_table_6,
    order_marketing[['order_id','campaign_id']],
    on = 'order_id',
    how = 'left'
)

fact_table_8 = pd.merge(
    order_item,
    fact_table_7,
    on = 'order_id',
    how = 'left',
    suffixes=['','_F7']
).drop(columns=['unit_price'])
fact_table_8

,order_id,line_number,product_id,quantity,discount_amount,tax_amount,customer_id,date_id,time_id,billing_address_id,channel_id,order_status,currency,shipping_amount,order_subtotal,order_discount,order_tax,order_total,campaign_id
0,100001,1,137,2,13623.34,38009.13,228,20250824,326,359.0,1,delivered,ARS,0.0,608021.07,37631.06,119781.90,690171.91,2.0
1,100001,2,148,2,2771.62,28519.99,228,20250824,326,359.0,1,delivered,ARS,0.0,608021.07,37631.06,119781.90,690171.91,2.0
2,100001,3,53,3,8387.43,33465.84,228,20250824,326,359.0,1,delivered,ARS,0.0,608021.07,37631.06,119781.90,690171.91,2.0
3,100001,4,77,3,12848.67,19786.95,228,20250824,326,359.0,1,delivered,ARS,0.0,608021.07,37631.06,119781.90,690171.91,2.0
4,100002,1,140,1,5071.28,7809.77,101,20250722,834,157.0,3,delivered,ARS,0.0,791961.63,42047.78,157481.91,907395.76,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
959,100399,1,81,3,0.00,45630.86,53,20250716,425,81.0,2,paid,ARS,0.0,739301.85,8544.03,153459.14,884216.96,6.0
960,100399,2,102,3,8544.03,87918.12,53,20250716,425,81.0,2,paid,ARS,0.0,739301.85,8544.03,153459.14,884216.96,6.0
961,100399,3,37,1,0.00,19910.17,53,20250716,425,81.0,2,paid,ARS,0.0,739301.85,8544.03,153459.14,884216.96,6.0
962,100400,1,127,1,2629.91,27061.78,13,20250612,1379,20.0,2,paid,ARS,0.0,160693.88,4089.83,32886.85,189490.90,NaN


In [24]:
products.query("product_id == 137")

,product_id,sku,product_name,brand,category_id,unit_price,unit_cost,active_from,active_to,created_at
136,137,SKU-00137,Norte Mochilas 137,Norte,10,97309.59,68624.94,2025-06-23,NaN,2025-06-23 04:58:47


In [26]:
fact_table_8.query("product_id == 55")

,order_id,line_number,product_id,quantity,discount_amount,tax_amount,customer_id,date_id,time_id,billing_address_id,channel_id,order_status,currency,shipping_amount,order_subtotal,order_discount,order_tax,order_total,campaign_id
195,100079,1,55,3,33191.71,62732.33,239,20250703,831,NaN,4,paid,ARS,0.0,629700.79,33308.81,125242.32,721634.30,12.0
385,100163,2,55,2,22127.81,41821.55,55,20250729,84,84.0,4,pending,ARS,0.0,351740.85,37783.34,65931.08,379888.59,11.0
566,100234,1,55,3,33191.71,62732.33,222,20250825,756,349.0,3,delivered,ARS,0.0,1146939.69,60993.11,228048.78,1313995.36,8.0
639,100262,2,55,2,22127.81,41821.55,173,20250727,749,265.0,3,pending,ARS,0.0,342602.16,24554.29,66790.05,384837.93,8.0
655,100268,2,55,1,5531.95,22072.49,210,20250618,149,328.0,3,delivered,ARS,0.0,550128.17,51247.35,104764.97,603645.80,NaN
745,100309,1,55,2,0.00,46468.39,85,20250716,444,129.0,1,paid,ARS,0.0,669812.74,22426.73,135951.06,783337.07,1.0
794,100327,2,55,1,11063.90,20910.78,150,20250829,177,NaN,4,created,ARS,0.0,169849.56,16649.91,32171.93,185371.57,11.0
934,100388,2,55,1,2212.78,22769.51,2,20250808,383,NaN,2,paid,ARS,0.0,449929.02,12432.44,91874.28,529370.86,NaN


In [50]:
dim_order = orders[[
    'order_id',
    # 'customer_id',
    'order_status',
    # 'order_date',
    # 'channel_id',
    # 'billing_address_id',
    # 'shipping_address_id',
    'currency',
    'shipping_amount',
    'order_subtotal',
    'order_discount',
    'order_tax',
    'order_total',
    # 'created_at',
    # 'date',
    # 'time'
]]

In [51]:
quantity_of_items = order_item.groupby('order_id')['line_number'].max().reset_index().rename(columns = {'line_number':'quantity_of_items'})
dim_order = pd.merge(
    dim_order,
    quantity_of_items,
    on = 'order_id',
    how='left'
)
dim_order

,order_id,order_status,currency,shipping_amount,order_subtotal,order_discount,order_tax,order_total,quantity_of_items
0,100001,delivered,ARS,0.0,608021.07,37631.06,119781.90,690171.91,4
1,100002,delivered,ARS,0.0,791961.63,42047.78,157481.91,907395.76,4
2,100003,paid,ARS,0.0,138581.10,16629.73,25609.79,147561.16,1
3,100004,shipped,ARS,0.0,125460.50,15055.26,23185.10,133590.34,1
4,100005,canceled,ARS,0.0,823628.91,82362.89,155665.86,896931.88,2
...,...,...,...,...,...,...,...,...,...
395,100396,delivered,ARS,0.0,255388.10,0.00,53631.50,309019.60,2
396,100397,delivered,ARS,0.0,412265.35,35573.92,79105.20,455796.63,2
397,100398,shipped,ARS,0.0,310203.67,13046.28,62403.05,359560.44,2
398,100399,paid,ARS,0.0,739301.85,8544.03,153459.14,884216.96,3


In [58]:
fact_table_9 = fact_table_8.drop(columns = ['line_number','discount_amount','tax_amount','order_status', 'currency', 'shipping_amount', 'order_subtotal','order_discount', 'order_tax', 'order_total'])
fact_table_9

,order_id,product_id,quantity,customer_id,date_id,time_id,billing_address_id,channel_id,campaign_id
0,100001,137,2,228,20250824,326,359.0,1,2.0
1,100001,148,2,228,20250824,326,359.0,1,2.0
2,100001,53,3,228,20250824,326,359.0,1,2.0
3,100001,77,3,228,20250824,326,359.0,1,2.0
4,100002,140,1,101,20250722,834,157.0,3,8.0
...,...,...,...,...,...,...,...,...,...
959,100399,81,3,53,20250716,425,81.0,2,6.0
960,100399,102,3,53,20250716,425,81.0,2,6.0
961,100399,37,1,53,20250716,425,81.0,2,6.0
962,100400,127,1,13,20250612,1379,20.0,2,NaN


In [60]:
dim_date.reset_index(drop=True)

,date_key,full_date,year,quarter,month,month_name,week_of_year,day_of_month,day_of_week,day_name,is_weekend
0,20250601,2025-06-01,2025,2,6,June,22,1,7,Sunday,True
1,20250602,2025-06-02,2025,2,6,June,23,2,1,Monday,False
2,20250603,2025-06-03,2025,2,6,June,23,3,2,Tuesday,False
3,20250604,2025-06-04,2025,2,6,June,23,4,3,Wednesday,False
4,20250605,2025-06-05,2025,2,6,June,23,5,4,Thursday,False
...,...,...,...,...,...,...,...,...,...,...,...
87,20250827,2025-08-27,2025,3,8,August,35,27,3,Wednesday,False
88,20250828,2025-08-28,2025,3,8,August,35,28,4,Thursday,False
89,20250829,2025-08-29,2025,3,8,August,35,29,5,Friday,False
90,20250830,2025-08-30,2025,3,8,August,35,30,6,Saturday,True


In [62]:
pd.read_csv("../warehouse/dim_product.csv")

,product_id,sku,product_name,brand,unit_price,unit_cost,active_from,active_to,created_at,category_name,parent_category_name
0,1,SKU-00001,Andina Remeras 001,Andina,112524.82,68777.76,2025-02-05,NaN,2025-02-05 03:43:54,Remeras,Ropa
1,2,SKU-00002,Lago Remeras 002,Lago,9608.49,5509.74,2025-03-01,NaN,2025-03-01 18:23:57,Remeras,Ropa
2,3,SKU-00003,Lago Zapatillas 003,Lago,36963.89,25775.53,2025-07-27,NaN,2025-07-27 00:14:11,Zapatillas,Calzado
3,4,SKU-00004,Lago Pantalones 004,Lago,54336.32,31997.02,2025-07-15,NaN,2025-07-15 12:15:18,Pantalones,Ropa
4,5,SKU-00005,Pampa Gorras 005,Pampa,57052.01,36284.45,2025-03-09,NaN,2025-03-09 01:34:55,Gorras,Accesorios
...,...,...,...,...,...,...,...,...,...,...,...
145,146,SKU-00146,Lago Pantalones 146,Lago,70778.34,50792.07,2025-07-10,NaN,2025-07-10 19:40:04,Pantalones,Ropa
146,147,SKU-00147,Lago Pantalones 147,Lago,99963.68,75814.78,2025-06-07,2025-07-27,2025-06-07 14:51:32,Pantalones,Ropa
147,148,SKU-00148,Norte Mochilas 148,Norte,69290.55,54575.92,2025-04-03,2025-07-21,2025-04-03 03:37:02,Mochilas,Accesorios
148,149,SKU-00149,Andina Mochilas 149,Andina,62730.25,37478.32,2025-02-01,NaN,2025-02-01 16:33:20,Mochilas,Accesorios


In [1]:
import pandas as pd
df = pd.read_csv("../staging/products_clean.csv")
df.head()

,product_id,sku,product_name,brand,unit_price,unit_cost,active_from,active_to,created_at,category_name,parent_category_name
0,1,SKU-00001,Andina Remeras 001,Andina,112524.82,68777.76,2025-02-05,NaN,2025-02-05 03:43:54,Remeras,Ropa
1,2,SKU-00002,Lago Remeras 002,Lago,9608.49,5509.74,2025-03-01,NaN,2025-03-01 18:23:57,Remeras,Ropa
2,3,SKU-00003,Lago Zapatillas 003,Lago,36963.89,25775.53,2025-07-27,NaN,2025-07-27 00:14:11,Zapatillas,Calzado
3,4,SKU-00004,Lago Pantalones 004,Lago,54336.32,31997.02,2025-07-15,NaN,2025-07-15 12:15:18,Pantalones,Ropa
4,5,SKU-00005,Pampa Gorras 005,Pampa,57052.01,36284.45,2025-03-09,NaN,2025-03-09 01:34:55,Gorras,Accesorios


In [22]:
import pandas as pd
customer_a = pd.read_csv("../raw/customer_addresses.csv")
orders = pd.read_csv("../raw/orders.csv")
orders

,order_id,customer_id,order_status,order_date,channel_id,billing_address_id,shipping_address_id,currency,shipping_amount,order_subtotal,order_discount,order_tax,order_total,created_at
0,100001,228,delivered,2025-08-24 05:25:58,1,359.0,358,ARS,0.0,608021.07,37631.06,119781.90,690171.91,2025-08-24 05:25:58
1,100002,101,delivered,2025-07-22 13:53:32,3,157.0,156,ARS,0.0,791961.63,42047.78,157481.91,907395.76,2025-07-22 13:53:32
2,100003,99,paid,2025-07-25 22:00:42,3,154.0,153,ARS,0.0,138581.10,16629.73,25609.79,147561.16,2025-07-25 22:00:42
3,100004,9,shipped,2025-08-30 13:25:10,1,NaN,14,ARS,0.0,125460.50,15055.26,23185.10,133590.34,2025-08-30 13:25:10
4,100005,108,canceled,2025-07-08 10:31:47,3,NaN,168,ARS,0.0,823628.91,82362.89,155665.86,896931.88,2025-07-08 10:31:47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,100396,159,delivered,2025-06-03 22:44:01,3,242.0,241,ARS,0.0,255388.10,0.00,53631.50,309019.60,2025-06-03 22:44:01
396,100397,17,delivered,2025-08-19 10:44:46,2,25.0,24,ARS,0.0,412265.35,35573.92,79105.20,455796.63,2025-08-19 10:44:46
397,100398,95,shipped,2025-06-24 10:41:53,3,147.0,146,ARS,0.0,310203.67,13046.28,62403.05,359560.44,2025-06-24 10:41:53
398,100399,53,paid,2025-07-16 07:04:31,2,81.0,80,ARS,0.0,739301.85,8544.03,153459.14,884216.96,2025-07-16 07:04:31


In [7]:
orders.head()

,order_id,customer_id,order_status,order_date,channel_id,billing_address_id,shipping_address_id,currency,shipping_amount,order_subtotal,order_discount,order_tax,order_total,created_at
0,100001,228,delivered,2025-08-24 05:25:58,1,359.0,358,ARS,0.0,608021.07,37631.06,119781.90,690171.91,2025-08-24 05:25:58
1,100002,101,delivered,2025-07-22 13:53:32,3,157.0,156,ARS,0.0,791961.63,42047.78,157481.91,907395.76,2025-07-22 13:53:32
2,100003,99,paid,2025-07-25 22:00:42,3,154.0,153,ARS,0.0,138581.10,16629.73,25609.79,147561.16,2025-07-25 22:00:42
3,100004,9,shipped,2025-08-30 13:25:10,1,NaN,14,ARS,0.0,125460.50,15055.26,23185.10,133590.34,2025-08-30 13:25:10
4,100005,108,canceled,2025-07-08 10:31:47,3,NaN,168,ARS,0.0,823628.91,82362.89,155665.86,896931.88,2025-07-08 10:31:47


In [6]:
important_cols = [
    'address_id',
    'city',
    'province',
    'country_code'
]

customer_a = customer_a[important_cols]
customer_a.head()

,address_id,city,province,country_code
0,1,Mendoza,Santa Fe,AR
1,2,Neuquén,CABA,AR
2,3,Salta,Neuquén,AR
3,4,Comodoro Rivadavia,Mendoza,AR
4,5,Salta,Neuquén,AR


In [12]:
important_cols = [
    'address_id',
    'city',
    'province',
    'country_code'
]

customer_a = customer_a[important_cols]
customer_a.head()

unique_locations = customer_a[["city","province","country_code"]].drop_duplicates()

unique_locations["location_id"] = range(1, len(unique_locations) + 1)

locations_df = pd.merge(customer_a, unique_locations, on = ["city","province","country_code"])

locations_mapping = locations_df[["address_id","location_id"]]
locations_df = (locations_df[["location_id","city","province","country_code"]]
                .drop_duplicates()
                .reset_index(drop=True)
                )
locations_df

,location_id,city,province,country_code
0,1,Mendoza,Santa Fe,AR
1,2,Neuquén,CABA,AR
2,3,Salta,Neuquén,AR
3,4,Comodoro Rivadavia,Mendoza,AR
4,5,Comodoro Rivadavia,Tucumán,AR
...,...,...,...,...
107,108,Rosario,Mendoza,AR
108,109,Comodoro Rivadavia,Buenos Aires,AR
109,110,Mendoza,Buenos Aires,AR
110,111,Neuquén,Entre Ríos,AR


In [15]:
# import pandas as pd

# def build_dim_location(customers: pd.DataFrame) -> tuple[pd.DataFrame, pd.DataFrame]:
#     """
#     Build a location dimension table and mapping from customer addresses.

#     Parameters
#     ----------
#     customers : pd.DataFrame
#         Raw customer dataframe. Must contain at least:
#         - address_id
#         - city
#         - province
#         - country_code

#     Returns
#     -------
#     locations_df : pd.DataFrame
#         Dimension table with surrogate location_id and unique locations.
    
#     location_mapping : pd.DataFrame
#         Mapping table between customer address_id and generated location_id.
#     """

# Columns required for the transformation
required_cols = ["address_id", "city", "province", "country_code"]
missing = [col for col in required_cols if col not in customer_a.columns]
if missing:
    raise ValueError(f"Missing required columns: {missing}")

# Keep only important columns
customers_trimmed = customer_a[required_cols].copy()

# Generate unique locations
unique_locations = (
    customers_trimmed[["city", "province", "country_code"]]
    .drop_duplicates()
    .reset_index(drop=True)
)
unique_locations.insert(0, "location_id", range(1, len(unique_locations) + 1))

# Join back to create mapping (address_id → location_id)
locations_with_keys = pd.merge(
    customers_trimmed,
    unique_locations,
    on=["city", "province", "country_code"],
    how="left"
)

location_mapping = locations_with_keys[["address_id", "location_id"]]

# Final dimension table (clean)
locations_df = (
    unique_locations[["location_id", "city", "province", "country_code"]]
    .sort_values("location_id")
    .reset_index(drop=True)
)


,address_id,location_id
0,1,1
1,2,2
2,3,3
3,4,4
4,5,3
...,...,...
389,390,91
390,391,31
391,392,32
392,393,70


In [1]:
import pandas as pd
orders = pd.read_csv("../raw/orders.csv")
order_items = pd.read_csv("../raw/order_items.csv")
orders.head()

,order_id,customer_id,order_status,order_date,channel_id,billing_address_id,shipping_address_id,currency,shipping_amount,order_subtotal,order_discount,order_tax,order_total,created_at
0,100001,228,delivered,2025-08-24 05:25:58,1,359.0,358,ARS,0.0,608021.07,37631.06,119781.90,690171.91,2025-08-24 05:25:58
1,100002,101,delivered,2025-07-22 13:53:32,3,157.0,156,ARS,0.0,791961.63,42047.78,157481.91,907395.76,2025-07-22 13:53:32
2,100003,99,paid,2025-07-25 22:00:42,3,154.0,153,ARS,0.0,138581.10,16629.73,25609.79,147561.16,2025-07-25 22:00:42
3,100004,9,shipped,2025-08-30 13:25:10,1,NaN,14,ARS,0.0,125460.50,15055.26,23185.10,133590.34,2025-08-30 13:25:10
4,100005,108,canceled,2025-07-08 10:31:47,3,NaN,168,ARS,0.0,823628.91,82362.89,155665.86,896931.88,2025-07-08 10:31:47


In [ ]:
orders

In [2]:
order_items.head()

,order_id,line_number,product_id,quantity,unit_price,discount_amount,tax_amount
0,100001,1,137,2,97309.59,13623.34,38009.13
1,100001,2,148,2,69290.55,2771.62,28519.99
2,100001,3,53,3,55916.19,8387.43,33465.84
3,100001,4,77,3,35690.74,12848.67,19786.95
4,100002,1,140,1,42260.66,5071.28,7809.77


In [3]:
import pandas as pd

campaigns = pd.read_csv("../raw/campaigns.csv")
orders = pd.read_csv("../raw/orders.csv")
location_customer_address_mapping = pd.read_csv("../staging/mappings/location_customer_address_mapping.csv", index_col=0)
campaigns.head()

,campaign_id,channel_id,campaign_name,utm_source,utm_medium,utm_campaign,start_date,end_date,budget_ars
0,1,1,Invierno 2025 Performance - Web,google,cpc,invierno_2025,2025-08-11,2025-08-20,4065939.23
1,2,1,Retargeting Julio 2025 - Web,meta,social,retargeting_jul_2025,2025-08-08,2025-08-27,3157411.81
2,3,1,Nuevo Drop Agosto 2025 - Web,meta,social,nuevo_drop_ago_2025,2025-07-01,2025-07-15,2965770.58
3,4,2,Invierno 2025 Performance - App,google,cpc,invierno_2025,2025-08-01,2025-08-10,2033213.71
4,5,2,Retargeting Julio 2025 - App,meta,social,retargeting_jul_2025,2025-08-15,2025-09-04,3052450.37


In [9]:
categories = pd.read_csv("../raw/categories.csv")
categories_clean = (
    categories.merge(categories.add_prefix("parent_"), on = "parent_category_id", how = "left")
              .drop(columns = ["parent_category_id","parent_parent_category_id"])
              .rename(columns = {"category_name": "category"})
)
customer_addresses = pd.read_csv("../raw/customer_addresses.csv")

important_columns = ["address_id", "city", "province", "country_code"]
customers_address_clean = customer_addresses[important_columns].copy()
customers_address_clean
customers_address_clean.to_csv("staging/customers_address_clean.csv", index=False)
print("customers_address_clean.csv created!")

,address_id,city,province,country_code
0,1,Mendoza,Santa Fe,AR
1,2,Neuquén,CABA,AR
2,3,Salta,Neuquén,AR
3,4,Comodoro Rivadavia,Mendoza,AR
4,5,Salta,Neuquén,AR
...,...,...,...,...
389,390,CABA,Córdoba,AR
390,391,La Plata,Neuquén,AR
391,392,Mar del Plata,Santa Fe,AR
392,393,Rosario,Córdoba,AR


In [19]:
from pandas import read_csv
customer_addresses_clean = read_csv("../staging/customer_addresses_clean.csv")
only_shipping = customer_addresses_clean["address_type"] == "shipping"
shipping_addresses = customer_addresses_clean[only_shipping]

unique_cities = shipping_addresses[["city", "province", "country_code"]].drop_duplicates()
unique_cities["city_id"] = range(1, len(unique_cities) + 1)

dim_location = unique_cities.copy()

address_city_mapping = (
    shipping_addresses.merge(unique_cities, on = ["city", "province", "country_code"], how = "left")
                      .drop(columns = ["city","province","country_code", "address_type"])
    )

,address_id,city_id
0,1,1
1,3,2
2,4,3
3,6,4
4,7,5
...,...,...
245,387,72
246,388,83
247,390,76
248,391,22


In [25]:
important_columns = [
    'order_id',
    'customer_id',
    'shipping_address_id',
    'channel_id',
    'order_status',
    'order_date'
]
orders_trimmed = orders[important_columns]

orders_paid = orders_trimmed[orders_trimmed["order_status"].isin(["paid","shipped","delivered"])]
orders_paid

,order_id,customer_id,shipping_address_id,channel_id,order_status,order_date
0,100001,228,358,1,delivered,2025-08-24 05:25:58
1,100002,101,156,3,delivered,2025-07-22 13:53:32
2,100003,99,153,3,paid,2025-07-25 22:00:42
3,100004,9,14,1,shipped,2025-08-30 13:25:10
5,100006,14,21,3,delivered,2025-08-19 00:59:38
...,...,...,...,...,...,...
395,100396,159,241,3,delivered,2025-06-03 22:44:01
396,100397,17,24,2,delivered,2025-08-19 10:44:46
397,100398,95,146,3,shipped,2025-06-24 10:41:53
398,100399,53,80,2,paid,2025-07-16 07:04:31


In [6]:
60*24*365

525600

In [4]:
import pandas as pd
import datetime as dt
orders = pd.read_csv("../raw/orders.csv")

orders["order_date"] = pd.to_datetime(orders["order_date"]).dt.floor("min")
orders["datetime_id"] = orders["order_date"].dt.strftime("%Y%m%d%H%M").astype(int)
orders
start_date = orders["order_date"].min()
end_date = orders["order_date"].max()
date_range = pd.date_range(start = start_date, end = end_date, freq="min")  # T = minuto
dim_datetime = pd.DataFrame({
    "time_key": range(1, len(date_range) + 1),
    "datetime_id": date_range.strftime("%Y%m%d%H%M").astype(int),  # PK estilo YYYYMMDDHHMM
    "date": date_range.date,
    "time": date_range.time,
    "year": date_range.year,
    "month": date_range.month,
    "month_name": date_range.strftime("%B"),
    "day": date_range.day,
    "day_of_week": date_range.dayofweek + 1,  # 1 = lunes
    "day_name": date_range.strftime("%A"),
    "hour": date_range.hour,
    "minute": date_range.minute,
    "is_weekend": date_range.dayofweek >= 5
})
dim_datetime

,time_key,datetime_id,date,time,year,month,month_name,day,day_of_week,day_name,hour,minute,is_weekend
0,1,202506010607,2025-06-01,06:07:00,2025,6,June,1,7,Sunday,6,7,True
1,2,202506010608,2025-06-01,06:08:00,2025,6,June,1,7,Sunday,6,8,True
2,3,202506010609,2025-06-01,06:09:00,2025,6,June,1,7,Sunday,6,9,True
3,4,202506010610,2025-06-01,06:10:00,2025,6,June,1,7,Sunday,6,10,True
4,5,202506010611,2025-06-01,06:11:00,2025,6,June,1,7,Sunday,6,11,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
131975,131976,202508312142,2025-08-31,21:42:00,2025,8,August,31,7,Sunday,21,42,True
131976,131977,202508312143,2025-08-31,21:43:00,2025,8,August,31,7,Sunday,21,43,True
131977,131978,202508312144,2025-08-31,21:44:00,2025,8,August,31,7,Sunday,21,44,True
131978,131979,202508312145,2025-08-31,21:45:00,2025,8,August,31,7,Sunday,21,45,True


In [35]:
orders_clean = pd.read_csv("../staging/orders_clean.csv")
order_marketing_clean = pd.read_csv("../staging/order_marketing_clean.csv")
order_items_clean = pd.read_csv("../staging/order_items_clean.csv")

In [45]:
orders_with_campaign_id = (
    orders_clean
    .merge(order_marketing_clean, on = "order_id", how = "inner")
)
fact_table = (
    order_items_clean.merge(orders_with_campaign_id, on = "order_id", how = "inner")
)
fact_table

,order_id,line_number,product_id,quantity,unit_price,discount_amount,tax_amount,customer_id,shipping_address_id,channel_id,order_status,order_date,campaign_id
0,100001,1,137,2,97309.59,13623.34,38009.13,228,358,1,delivered,2025-08-24 05:25:58,2.0
1,100001,2,148,2,69290.55,2771.62,28519.99,228,358,1,delivered,2025-08-24 05:25:58,2.0
2,100001,3,53,3,55916.19,8387.43,33465.84,228,358,1,delivered,2025-08-24 05:25:58,2.0
3,100001,4,77,3,35690.74,12848.67,19786.95,228,358,1,delivered,2025-08-24 05:25:58,2.0
4,100002,1,140,1,42260.66,5071.28,7809.77,101,156,3,delivered,2025-07-22 13:53:32,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
526,100398,1,6,3,16426.03,0.00,10348.40,95,146,3,shipped,2025-06-24 10:41:53,7.0
527,100398,2,121,2,130462.79,13046.28,52054.65,95,146,3,shipped,2025-06-24 10:41:53,7.0
528,100399,1,81,3,72429.93,0.00,45630.86,53,80,2,paid,2025-07-16 07:04:31,6.0
529,100399,2,102,3,142400.58,8544.03,87918.12,53,80,2,paid,2025-07-16 07:04:31,6.0


In [43]:
orders_with_campaign_id = orders_clean.merge(order_marketing_clean, on = "order_id", how = "inner")
fact_item_sell = order_items_clean.merge(orders_with_campaign_id, on = "order_id", how = "inner")
fact_item_sell

,order_id,line_number,product_id,quantity,unit_price,discount_amount,tax_amount,customer_id,shipping_address_id,channel_id,order_status,order_date,campaign_id
0,100001,1,137,2,97309.59,13623.34,38009.13,228,358,1,delivered,2025-08-24 05:25:58,2.0
1,100001,2,148,2,69290.55,2771.62,28519.99,228,358,1,delivered,2025-08-24 05:25:58,2.0
2,100001,3,53,3,55916.19,8387.43,33465.84,228,358,1,delivered,2025-08-24 05:25:58,2.0
3,100001,4,77,3,35690.74,12848.67,19786.95,228,358,1,delivered,2025-08-24 05:25:58,2.0
4,100002,1,140,1,42260.66,5071.28,7809.77,101,156,3,delivered,2025-07-22 13:53:32,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
526,100398,1,6,3,16426.03,0.00,10348.40,95,146,3,shipped,2025-06-24 10:41:53,7.0
527,100398,2,121,2,130462.79,13046.28,52054.65,95,146,3,shipped,2025-06-24 10:41:53,7.0
528,100399,1,81,3,72429.93,0.00,45630.86,53,80,2,paid,2025-07-16 07:04:31,6.0
529,100399,2,102,3,142400.58,8544.03,87918.12,53,80,2,paid,2025-07-16 07:04:31,6.0


In [47]:
import pandas as pd
fact = pd.read_csv("../warehouse/fact_order_item.csv")
dim_prod = pd.read_csv("../warehouse/dim_product.csv")
asd = fact.merge(dim_prod, on = "product_key", how = "left", suffixes=["","_dim"])
(asd.unit_price == asd.unit_price_dim).value_counts()

True    531
Name: count, dtype: int64

In [41]:
important_cols = [
    'order_id',
    'customer_id',
    'channel_id',
    'location_id',
    'order_date',
    'order_status',
    'currency',
    'shipping_amount',
    'order_subtotal',
    'order_discount',
    'order_tax',
    'order_total'
]

orders_paid_location = pd.merge(
    orders_paid,
    location_customer_address_mapping,
    left_on = "shipping_address_id",
    right_on = "address_id",
    how = "left"
)

orders_paid_location

,order_id,customer_id,order_status,order_date,channel_id,billing_address_id,shipping_address_id,currency,shipping_amount,order_subtotal,order_discount,order_tax,order_total,created_at,address_id,location_id
0,100001,228,delivered,2025-08-24 05:25:58,1,359.0,358,ARS,0.0,608021.07,37631.06,119781.90,690171.91,2025-08-24 05:25:58,358,60
1,100002,101,delivered,2025-07-22 13:53:32,3,157.0,156,ARS,0.0,791961.63,42047.78,157481.91,907395.76,2025-07-22 13:53:32,156,62
2,100003,99,paid,2025-07-25 22:00:42,3,154.0,153,ARS,0.0,138581.10,16629.73,25609.79,147561.16,2025-07-25 22:00:42,153,40
3,100004,9,shipped,2025-08-30 13:25:10,1,NaN,14,ARS,0.0,125460.50,15055.26,23185.10,133590.34,2025-08-30 13:25:10,14,13
4,100006,14,delivered,2025-08-19 00:59:38,3,NaN,21,ARS,0.0,229681.20,11338.91,45851.88,264194.17,2025-08-19 00:59:38,21,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,100396,159,delivered,2025-06-03 22:44:01,3,242.0,241,ARS,0.0,255388.10,0.00,53631.50,309019.60,2025-06-03 22:44:01,241,83
302,100397,17,delivered,2025-08-19 10:44:46,2,25.0,24,ARS,0.0,412265.35,35573.92,79105.20,455796.63,2025-08-19 10:44:46,24,23
303,100398,95,shipped,2025-06-24 10:41:53,3,147.0,146,ARS,0.0,310203.67,13046.28,62403.05,359560.44,2025-06-24 10:41:53,146,84
304,100399,53,paid,2025-07-16 07:04:31,2,81.0,80,ARS,0.0,739301.85,8544.03,153459.14,884216.96,2025-07-16 07:04:31,80,58


In [47]:
campaigns = pd.read_csv("../staging/campaigns_clean.csv")
order_marketing = pd.read_csv("../raw/order_marketing.csv")
order_marketing.head()

,order_id,channel_id,campaign_id,utm_source,utm_medium,utm_campaign,utm_content,utm_term
0,100001,1,2.0,meta,social,retargeting_jul_2025,NaN,mochilas
1,100002,3,8.0,meta,social,retargeting_jul_2025,remarketing,remeras
2,100003,3,9.0,meta,social,nuevo_drop_ago_2025,remarketing,camperas
3,100004,1,NaN,email,email,newsletter,NaN,camperas
4,100005,3,9.0,meta,social,nuevo_drop_ago_2025,promo,mochilas


In [54]:
order_marketing = pd.read_csv("../raw/order_marketing.csv")
order_marketing_campaign = order_marketing[~ order_marketing["campaign_id"].isnull()]
important_cols = [
    "order_id",
    "campaign_id"
]
order_marketing_campaign_mapping = order_marketing_campaign[important_cols]

,order_id,campaign_id
0,100001,2.0
1,100002,8.0
2,100003,9.0
4,100005,9.0
5,100006,7.0
...,...,...
394,100395,9.0
395,100396,7.0
396,100397,4.0
397,100398,7.0


In [51]:
campaigns.head()

,campaign_id,campaign_name,utm_source,utm_medium,utm_campaign,start_date,end_date,budget_ars
0,1,Invierno 2025 Performance - Web,google,cpc,invierno_2025,2025-08-11,2025-08-20,4065939.23
1,2,Retargeting Julio 2025 - Web,meta,social,retargeting_jul_2025,2025-08-08,2025-08-27,3157411.81
2,3,Nuevo Drop Agosto 2025 - Web,meta,social,nuevo_drop_ago_2025,2025-07-01,2025-07-15,2965770.58
3,4,Invierno 2025 Performance - App,google,cpc,invierno_2025,2025-08-01,2025-08-10,2033213.71
4,5,Retargeting Julio 2025 - App,meta,social,retargeting_jul_2025,2025-08-15,2025-09-04,3052450.37
